In [ ]:
!pip install googlesearch-python

from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re
import json
import time
from IPython.display import FileLink

def limpiar_nombre(raw_title):
    if not raw_title:
        return "Nombre no encontrado"
    nombre = raw_title.strip()

    # Palabras a eliminar comunes en títulos
    palabras_eliminar = [
        r'\bcontacto(s)?\b',
        r'\bpágina oficial\b',
        r'\bcontact\b',
        r'\bhome\b',
        r'\bportal\b',
        r'\bweb\b',
        r'\bsite\b',
        r'\boficial\b',
        r'[-|–|—|:|•].*',  # eliminar todo después de guiones, dos puntos, etc.
        r'\d{1,4}[\s,.-]?[a-zA-Z]*'  # eliminar posibles números y direcciones cortas
    ]

    nombre_limpio = nombre.lower()
    for palabra in palabras_eliminar:
        nombre_limpio = re.sub(palabra, '', nombre_limpio)

    nombre_limpio = re.sub(r'\s+', ' ', nombre_limpio).strip()

    # Capitalizar cada palabra
    nombre_limpio = ' '.join(w.capitalize() for w in nombre_limpio.split())

    if not nombre_limpio:
        return "Nombre no encontrado"
    return nombre_limpio

busquedas = [
    "colegio contacto correo site:.ec",
    "colegio privado Quito correo contacto",
    "colegio católico Guayaquil contacto email",
    "institución educativa correo site:.ec",
    "colegios Ecuador email contacto",
    "escuelas Ecuador correo contacto",
    "unidad educativa contacto email site:.ec",
    "colegio particular contacto correo",
    "colegio oficial correo Ecuador",
]

resultados = []
correos_vistos = set()
MAX_RESULTADOS = 1000

for query in busquedas:
    print(f"Buscando: {query}")
    try:
        for url in search(query, num_results=20, lang="es"):
            if len(resultados) >= MAX_RESULTADOS:
                break
            try:
                res = requests.get(url, timeout=10)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")
                texto = soup.get_text(separator=' ')
                correos = set(re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", texto))
                correos = {c.lower() for c in correos if c.lower() not in correos_vistos}

                if not correos:
                    continue

                nombre_raw = soup.title.string if soup.title else None
                nombre = limpiar_nombre(nombre_raw)

                correos_vistos.update(correos)

                resultado = {
                    "nombre": nombre,
                    "correos": ", ".join(sorted(correos))
                }
                resultados.append(resultado)
                print(f"  + {nombre}: {len(correos)} correos")
                time.sleep(1)
            except Exception as e:
                print(f"Error en {url}: {e}")
    except Exception as e:
        print(f"Error en búsqueda '{query}': {e}")
    if len(resultados) >= MAX_RESULTADOS:
        break

print(f"\nTotal resultados: {len(resultados)}")

# Guardar en JSON
nombre_archivo = "colegios_ecuador.json"
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write("[\n")
    for i, item in enumerate(resultados):
        coma = "," if i < len(resultados)-1 else ""
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(f"  {json_str}{coma}\n")
    f.write("]\n")

print(f"\nArchivo '{nombre_archivo}' guardado con éxito.")

# Mostrar enlace para descargar en Colab
display(FileLink(nombre_archivo))

# Mostrar una muestra de los resultados en consola (solo 10)
print("\nEjemplo de resultados (10 primeros):")
for item in resultados[:10]:
    print(json.dumps(item, ensure_ascii=False, indent=2))
